In [2]:
from fairness.matching import compute
import pandas as pd
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score,confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeClassifier
import numpy as np
import xgboost as xgb
import matplotlib.pyplot as plt

In [3]:
dataset = pd.read_csv('./dataset/student-por.csv', sep=';')

In [4]:
dataset.drop(columns=['school', 'address', 'G1', 'G2'], inplace=True)

In [6]:
from fairness.pre_processing import categorical_to_numeric_converter
from fairness.pre_processing import fix_protected_attributes


dataset = categorical_to_numeric_converter(dataset)
dataset = fix_protected_attributes(dataset, ['G3'])

In [7]:
output_column = "G3"

In [8]:
X = dataset.iloc[:,:len(dataset.columns) - 2]
y = dataset.loc[:, output_column:]

In [9]:
estimator = DecisionTreeClassifier()

parameters = {
    'max_depth': [2, 3, 5, 10, 20],
    'min_samples_leaf': [5, 10, 20, 50, 100],
    'criterion': ["gini", "entropy"]
}

In [10]:
grid_search = GridSearchCV(
    estimator=estimator,
    param_grid=parameters,
    n_jobs=10,
    cv=10,
    verbose=True
)

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [12]:
standard_scaler_train = StandardScaler()
X_train = standard_scaler_train.fit_transform(X_train)

standard_scaler_test = StandardScaler()
X_test = standard_scaler_test.fit_transform(X_test)

In [13]:
y

,G3
0,11
1,11
2,12
3,14
4,13
...,...
644,10
645,16
646,9
647,10


In [13]:
grid_search_classifier = grid_search.fit(X_train, y_train)

Fitting 10 folds for each of 50 candidates, totalling 500 fits


C:\Users\anton\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\model_selection\_split.py:700: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=10.
  warnings.warn(


In [14]:
y_pred = grid_search_classifier.predict(X_test)

In [15]:
print(accuracy_score(y_test, y_pred))

0.20689655172413793


In [16]:
#cm_display = ConfusionMatrixDisplay(confusion_matrix(y_test, y_pred), display_labels=[False, True])
#cm_display.plot()
#plt.show()